In [1]:
%matplotlib inline
import math
import time
import numpy as np
import tensorflow as tf
import random

from d2l import tensorflow as d2l

In [2]:
class Timer:  #@save
    """Record multiple running times."""
    def __init__(self):
        self.times = []
        self.start()

    def start(self):
        """Start the timer."""
        self.tik = time.time()

    def stop(self):
        """Stop the timer and record the time in a list."""
        self.times.append(time.time() - self.tik)
        return self.times[-1]

    def avg(self):
        """Return the average time."""
        return sum(self.times) / len(self.times)

    def sum(self):
        """Return the sum of time."""
        return sum(self.times)

    def cumsum(self):
        """Return the accumulated time."""
        return np.array(self.times).cumsum().tolist()

### Vectorization

In [3]:
n = 10000
a = tf.ones(n)
b = tf.ones(n)

In [4]:
a

<tf.Tensor: shape=(10000,), dtype=float32, numpy=array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)>

In [5]:
c = tf.Variable(tf.zeros(n))
timer = Timer()
for i in range(n):
    c[i].assign(a[i] + b[i])
    
f'{timer.stop():.5f} sec'

'3.57890 sec'

In [6]:
timer.start()
d = a+b
f'{timer.stop():.5f} sec'

'0.00027 sec'

### Generating the Dataset

In [7]:
def synthetic_data(w, b, num_examples):  #@save
    """Generate y = Xw + b + noise."""
    
    # Initial zero vector with length = num_examples 
    # Random x with normal distribution
    X = tf.zeros((num_examples, w.shape[0]))
    X += tf.random.normal(shape=X.shape)

    # Matrix multiply with constant 
    y = tf.matmul(X, tf.reshape(w, (-1, 1))) + b
    y += tf.random.normal(shape=y.shape, stddev=0.01)
    y = tf.reshape(y, (-1, 1))
    return X, y

true_w = tf.constant([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)

In [8]:
print('features:', features[0],'\nlabel:', labels[0])

features: tf.Tensor([-0.08559055 -0.5360829 ], shape=(2,), dtype=float32) 
label: tf.Tensor([5.8617077], shape=(1,), dtype=float32)


In [9]:
import plotly.express as px

In [10]:
px.scatter(x = features[:,1].numpy(), y =labels[:,0].numpy(),
           width = 500, height = 500,
           title = 'Feature 2 with Label')

In [11]:
px.scatter(x = features[:,0].numpy(), y =labels[:,0].numpy(),
           width = 500, height = 500,
           title = 'Feature 1 with Label')

### Reading the Dataset

Implement grabbing one minibatch of data at a time.
- This function takes a batch size, matrix of features and vector of labels

In [12]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # The examples are read at random, in no particular order
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        j = tf.constant(indices[i: min(i + batch_size, num_examples)])
        yield tf.gather(features, j), tf.gather(labels, j)

In [13]:
batch_size = 10

for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y)

tf.Tensor(
[[ 0.2955585   1.3211198 ]
 [-1.770104    0.43469474]
 [ 0.16181633  0.08531206]
 [-0.9267397  -0.9571995 ]
 [ 0.93018574 -0.37605357]
 [-0.34304872 -0.7871578 ]
 [-0.576506   -0.41528293]
 [-0.25411192 -0.20353481]
 [ 0.6028147  -0.00270418]
 [-1.8495697   0.8397488 ]], shape=(10, 2), dtype=float32) 
 tf.Tensor(
[[ 0.2969509]
 [-0.7971988]
 [ 4.2375913]
 [ 5.618064 ]
 [ 7.337027 ]
 [ 6.1880264]
 [ 4.478739 ]
 [ 4.3738813]
 [ 5.4160304]
 [-2.3689473]], shape=(10, 1), dtype=float32)
tf.Tensor(
[[-0.3455485   0.05917535]
 [ 0.81550324  0.36381134]
 [-0.858939    0.46401766]
 [-0.42717955  1.548035  ]
 [-1.2256192   0.23890634]
 [-0.61399144  0.76971215]
 [ 2.6594417  -0.6524099 ]
 [ 1.0188454   0.6772925 ]
 [-1.0330085   0.33971825]
 [ 0.3576284  -0.67700166]], shape=(10, 2), dtype=float32) 
 tf.Tensor(
[[ 3.3244052 ]
 [ 4.5963836 ]
 [ 0.90225357]
 [-1.9168922 ]
 [ 0.9402798 ]
 [ 0.37592196]
 [11.753383  ]
 [ 3.9387949 ]
 [ 0.96085   ]
 [ 7.2038436 ]], shape=(10, 1), dtype=flo

### Defining the Model

In [14]:
def linreg(X, w, b): #@save
    '''Linear Regression Model'''
    return tf.matmul(X,w) + b

### Defining the Loss Function

In [15]:
def squared_loss(y_hat, y): #@save
    '''Squared Loss'''
    return (y_hat - tf.reshape(y, y_hat.shape))**2/2

### Optimization Algorithm
Minibatch Stochastic Gradient Descent

In [16]:
def sgd(params, grads, lr, batch_size): #@save
    '''Minibatch Stochastic Gradient Descent'''
    for param, grad in zip(params, grads):
        param.assign_sub(lr*grad/batch_size)

### Training

In [17]:
# Parameters randomly initialize
w = tf.Variable(tf.random.normal(shape = (2, 1), mean = 0, stddev = 0.01))
b = tf.Variable(tf.zeros(1), trainable= True)

In [18]:
params_1 = [w, b]

dw = tf.constant([[-17.272],
                  [85.580376]])
db = tf.constant([-54.097393])
grad_1 = [dw, db]

for param, grad in zip(params_1 , grad_1):
    print(param)
    print('')
    print(grad)
    param.assign_sub(0.03*grad/10)
    print('Update parameters')
    print(param)
    print('end loop')
    print('')

<tf.Variable 'Variable:0' shape=(2, 1) dtype=float32, numpy=
array([[-0.01218541],
       [-0.0050498 ]], dtype=float32)>

tf.Tensor(
[[-17.272   ]
 [ 85.580376]], shape=(2, 1), dtype=float32)
Update parameters
<tf.Variable 'Variable:0' shape=(2, 1) dtype=float32, numpy=
array([[ 0.03963058],
       [-0.2617909 ]], dtype=float32)>
end loop

<tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>

tf.Tensor([-54.097393], shape=(1,), dtype=float32)
Update parameters
<tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([0.16229217], dtype=float32)>
end loop



In [19]:
lr = 0.03 # learning rate
num_epochs = 3
batch_size = 10
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        # Use gradient tape on loss functions
        with tf.GradientTape() as g:
            l = loss(net(X, w, b), y) # Minibatch Loss in 'X' and 'y'
        dw, db = g.gradient(l, [w, b])
        sgd([w, b], [dw, db], lr, batch_size)
    train_l = loss(net(features, w, b), labels)
    print(f'epoch {epoch + 1}, loss {float(tf.reduce_mean(train_l)):f}')

epoch 1, loss 0.043831
epoch 2, loss 0.000201
epoch 3, loss 0.000052


In [20]:
print(f'error in estimation w : {true_w - tf.reshape(w, true_w.shape)}')
print(f'error in estimation b : {true_b - b}')

error in estimation w : [-0.00033045 -0.00114632]
error in estimation b : [0.00056648]
